In [ ]:
!nvidia-smi

Wed Dec 17 15:00:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## SETUP

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Subset, Dataset
import os
import random
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [5]:
data_dir = "/content/drive/MyDrive/Bark images 2"

In [6]:

def list_samples_grouped_by_tree(root_dir, exts={".jpg",".jpeg",".png",".bmp",".webp"}):
    root = Path(root_dir)
    classes = sorted([p.name for p in root.iterdir() if p.is_dir()])
    class_to_idx = {c:i for i,c in enumerate(classes)}

    samples = []  # (img_path, label_idx, group_id)
    for cls in classes:
        cls_dir = root / cls
        tree_ids = sorted([p.name for p in cls_dir.iterdir() if p.is_dir()])
        for tree_id in tree_ids:
            tree_dir = cls_dir / tree_id
            group_id = f"{cls}/{tree_id}"  # unique per tree within class
            for img_path in tree_dir.rglob("*"):
                if img_path.is_file() and img_path.suffix.lower() in exts:
                    samples.append((str(img_path), class_to_idx[cls], group_id))
    return samples, classes, class_to_idx

def split_by_tree_within_each_class(samples, test_ratio=0.2, seed=42):
    """
    For each class, split its tree-folders into train/test.
    Guarantees no leakage across splits within the same tree folder.
    """
    rng = random.Random(seed)

    # collect groups per class
    by_class = {}
    for path, y, group in samples:
        by_class.setdefault(y, {}).setdefault(group, []).append((path, y, group))

    train, test = [], []
    for y, groups_dict in by_class.items():
        groups = list(groups_dict.keys())
        rng.shuffle(groups)

        n_test = max(1, int(round(len(groups) * test_ratio))) if len(groups) > 1 else 0
        test_groups = set(groups[:n_test])

        for g, items in groups_dict.items():
            (test if g in test_groups else train).extend(items)

    rng.shuffle(train)
    rng.shuffle(test)
    return train, test

class BarkDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, y, _group = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, y

# --- build everything ---
samples, class_names, class_to_idx = list_samples_grouped_by_tree(data_dir)
train_samples, test_samples = split_by_tree_within_each_class(samples, test_ratio=0.2, seed=42)


train_tfms = transforms.Compose([
    transforms.Resize(450),
    transforms.CenterCrop(450),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
])

test_tfms = transforms.Compose([
    transforms.Resize(450),
    transforms.CenterCrop(450),
    transforms.ToTensor(),
])

train_ds = BarkDataset(train_samples, transform=train_tfms)
test_ds  = BarkDataset(test_samples,  transform=test_tfms)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

print("Classes:", class_names)
print("Train images:", len(train_ds), "Test images:", len(test_ds))

Classes: ['acacia', 'apple', 'candle', 'cherry', 'cypress', 'fig', 'katsura', 'linden', 'magnolia', 'oak', 'olive', 'palm', 'pine', 'plane', 'red-gum', 'sakura']
Train images: 870 Test images: 256


### SANITY CHECK

In [7]:
train_groups = {g for _,_,g in train_samples}
test_groups  = {g for _,_,g in test_samples}
assert train_groups.isdisjoint(test_groups), "Leakage: same tree folder is in both train and test!"
print("No leakage across tree folders.")


No leakage across tree folders.


### MODEL AND CHECKPOINTS

In [ ]:
ckpt_dir = "/content/drive/MyDrive/bark_checkpoints/resnet50-pretrained"
os.makedirs(ckpt_dir, exist_ok=True)


In [9]:
import os, torch

def save_checkpoint(path, *, epoch, model, optimizer, scheduler=None, scaler=None,
                    best_metric=None, class_names=None, extra=None):
    ckpt = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_metric": best_metric,
        "class_names": class_names,
        "extra": extra or {},
        "torch_version": torch.__version__,
    }
    if scheduler is not None:
        ckpt["scheduler_state"] = scheduler.state_dict()
    if scaler is not None:
        ckpt["scaler_state"] = scaler.state_dict()
    torch.save(ckpt, path)

def load_checkpoint(path, model, optimizer=None, scheduler=None, scaler=None, map_location="cpu"):
    ckpt = torch.load(path, map_location=map_location, weights_only=False)
    model.load_state_dict(ckpt["model_state"])
    if optimizer is not None and "optimizer_state" in ckpt:
        optimizer.load_state_dict(ckpt["optimizer_state"])
    if scheduler is not None and "scheduler_state" in ckpt:
        scheduler.load_state_dict(ckpt["scheduler_state"])
    if scaler is not None and "scaler_state" in ckpt:
        scaler.load_state_dict(ckpt["scaler_state"])
    start_epoch = ckpt.get("epoch", -1) + 1
    best_metric = ckpt.get("best_metric", None)
    return start_epoch, best_metric, ckpt


# DELETES CHECKPOINTS

In [ ]:
# import os

# last_path = os.path.join(ckpt_dir, "last.pt")
# best_path = os.path.join(ckpt_dir, "best.pt")

# for p in [last_path, best_path]:
#     if os.path.exists(p):
#         os.remove(p)
# print("Deleted old checkpoints. Fresh run guaranteed.")

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# weights=None => random init (fresh)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Set classifier head for your 15 classes
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

Device: cuda
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]


In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

def make_loaders(input_size, batch_size=32):
    
    train_tfms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
        transforms.ToTensor(),
    ])
    test_tfms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ])

    train_ds = BarkDataset(train_samples, transform=train_tfms)
    test_ds  = BarkDataset(test_samples,  transform=test_tfms)

    # small batch size for bigger input
    if input_size >= 384:
        batch_size = min(batch_size, 16)
    if input_size >= 450:
        batch_size = min(batch_size, 8)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, test_loader

def train_one(config, run_dir):
    run_dir = Path(run_dir)
    run_dir.mkdir(parents=True, exist_ok=True)

    # Save config for reproducibility
    with open(run_dir/"config.json", "w") as f:
        json.dump(config, f, indent=2)

    # Model (scratch)
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(model.fc.in_features, len(class_names))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=config["label_smoothing"])
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=config["lr"],
                                momentum=0.9,
                                weight_decay=config["weight_decay"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config["epochs"])

    scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

    train_loader, test_loader = make_loaders(config["input_size"])

    best_acc = 0.0
    best_epoch = -1

    def run_epoch(loader, train: bool):
        model.train(train)
        total_loss, correct, total = 0.0, 0, 0

        for x, y in loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

            with torch.set_grad_enabled(train):
                with torch.cuda.amp.autocast(enabled=(device=="cuda")):
                    logits = model(x)
                    loss = criterion(logits, y)

                if train:
                    optimizer.zero_grad(set_to_none=True)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

            total_loss += float(loss.item()) * x.size(0)
            correct += int((logits.argmax(1) == y).sum().item())
            total += int(x.size(0))

        return total_loss / max(total, 1), correct / max(total, 1)

    history = []
    for epoch in range(config["epochs"]):
        tr_loss, tr_acc = run_epoch(train_loader, True)
        va_loss, va_acc = run_epoch(test_loader, False)
        scheduler.step()

        history.append({
            "epoch": epoch,
            "train_loss": tr_loss, "train_acc": tr_acc,
            "val_loss": va_loss, "val_acc": va_acc,
        })

        # Save last each epoch
        save_checkpoint(str(run_dir/"last.pt"),
                        epoch=epoch, model=model, optimizer=optimizer,
                        scheduler=scheduler, scaler=scaler,
                        best_metric=best_acc, class_names=class_names,
                        extra=history[-1])

        if va_acc > best_acc:
            best_acc, best_epoch = va_acc, epoch
            save_checkpoint(str(run_dir/"best.pt"),
                            epoch=epoch, model=model, optimizer=optimizer,
                            scheduler=scheduler, scaler=scaler,
                            best_metric=best_acc, class_names=class_names,
                            extra=history[-1])

    with open(run_dir/"history.json", "w") as f:
        json.dump(history, f, indent=2)

    return {"best_val_acc": best_acc, "best_epoch": best_epoch, "final_val_acc": history[-1]["val_acc"]}


In [12]:
# Loss/optim
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)  # set T_max=epochs
scaler = torch.amp.GradScaler(enabled=(device=="cuda"))

def run_epoch(loader, epoch, num_epochs, train: bool):
    model.train(train)
    total_loss, correct, total = 0.0, 0, 0

    for x, y in tqdm(loader, desc=f"Epoch {epoch+1:03d}/{num_epochs} [{"train" if train else "test"}]"):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

        with torch.set_grad_enabled(train):
            with torch.amp.autocast('cuda', enabled=(device=="cuda")):
                logits = model(x)
                loss = criterion(logits, y)

            if train:
                optimizer.zero_grad(set_to_none=True)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

        total_loss += float(loss.item()) * x.size(0)
        preds = logits.argmax(dim=1)
        correct += int((preds == y).sum().item())
        total += int(x.size(0))

    return total_loss / max(total, 1), correct / max(total, 1)

# Checkpoint paths
last_path = os.path.join(ckpt_dir, "last.pt")
best_path = os.path.join(ckpt_dir, "best.pt")

# Resume (if last exists)
start_epoch, best_acc = 0, 0.0
if os.path.exists(last_path):
    start_epoch, best_acc, _ = load_checkpoint(
        last_path, model, optimizer=optimizer, scheduler=scheduler, scaler=scaler, map_location=device
    )
    if best_acc is None:
        best_acc = 0.0
    print(f"Resumed from epoch {start_epoch}, best_acc={best_acc:.4f}")

# Train
epochs = 200
for epoch in range(start_epoch, epochs):
    train_loss, train_acc = run_epoch(train_loader, epoch, epochs, train=True)
    val_loss, val_acc = run_epoch(test_loader, epoch, epochs, train=False)

    scheduler.step()

    # Always save "last"
    save_checkpoint(
        last_path,
        epoch=epoch,
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        scaler=scaler,
        best_metric=best_acc,
        class_names=class_names,
        extra={"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc},
    )

    # Save "best" by validation accuracy
    if val_acc > best_acc:
        best_acc = val_acc
        save_checkpoint(
            best_path,
            epoch=epoch,
            model=model,
            optimizer=optimizer,
            scheduler=scheduler,
            scaler=scaler,
            best_metric=best_acc,
            class_names=class_names,
            extra={"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc},
        )

    print(f"Epoch {epoch+1:03d}/{epochs} | "
          f"train: loss={train_loss:.4f} acc={train_acc:.4f} | "
          f"val: loss={val_loss:.4f} acc={val_acc:.4f} | best={best_acc:.4f}")


Epoch 001/200 [test]: 100%|██████████| 32/32 [01:02<00:00,  1.95s/it]


Epoch 001/200 | train: loss=1.7244 acc=0.5736 | val: loss=1.8381 acc=0.5742 | best=0.5742


Epoch 002/200 [test]: 100%|██████████| 32/32 [00:04<00:00,  7.59it/s]


Epoch 002/200 | train: loss=1.0466 acc=0.8264 | val: loss=1.5226 acc=0.6797 | best=0.6797


Epoch 003/200 [test]: 100%|██████████| 32/32 [00:03<00:00,  9.68it/s]


...

Epoch 198/200 | train: loss=0.5942 acc=0.9989 | val: loss=1.4032 acc=0.7148 | best=0.8359


Epoch 199/200 [test]: 100%|██████████| 32/32 [00:04<00:00,  6.81it/s]


Epoch 199/200 | train: loss=0.5801 acc=1.0000 | val: loss=1.2627 acc=0.7695 | best=0.8359


Epoch 200/200 [test]: 100%|██████████| 32/32 [00:04<00:00,  6.93it/s]


Epoch 200/200 | train: loss=0.5785 acc=1.0000 | val: loss=1.3233 acc=0.7344 | best=0.8359
